In [3]:
from __future__ import absolute_import, division

import Model
import numpy as np 
#import preTool
from keras.preprocessing.sequence import pad_sequences
import senteval
import pickle
import copy
import operator

PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

maxLength = 30    
wordDim = 300    # 300 
num_neurons = [600,600]   # [600,600f]
extractLayer = ['X_input_second','X_input_mask_2']
kernel_reg = 0.001
batch_size = 400
merge_mode = 'concat'
checkpoint = [ "../Checkpoint_summary/Fre_2500-loss-1.3988-cata_acc-0.7049-val_loss-1.5115-cate_acc_val-0.6838_.hdf5"  ]# Change 
embeddingPath = "../Embedding/word2vec.810B.300d.txt"  # Change 
processedPath = "../processed/"
tokenizerFile = 'tokenizer.txt'
embeddingFile =  'embedding_matrix.txt'
dicFile = 'wordDic.txt'
marked_token = "maskedtoken"
trainable = True

model = []


# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [' '.join(s) for s in batch]
    
    with open(processedPath + tokenizerFile , "rb") as input_file:
        t = pickle.load(input_file)
    
    with open(processedPath + embeddingFile , "rb") as input_file:
        embedding_matrix = pickle.load(input_file)
  
    with open(processedPath + dicFile , "rb") as input_file:
        wholeVocab = pickle.load(input_file) 

    #embedding_matrix = np.zeros([49985,wordDim])
        
    sentences  = Model.preprocessing(sentences)
    sentences = t.texts_to_sequences(sentences)
    sentences = pad_sequences(sentences, maxlen=maxLength, padding='post')
    
    if len(model) == 0 :
        
        for weight in checkpoint:
            model_x = Model.buildModel(sentences,kernel_reg,num_neurons,merge_mode, 678318,maxLength, wordDim,np.zeros([678318,300]),trainable)
            print("load weights")
            model_x.load_weights(weight)
            model.append(model_x)

    output = []
    for i in model:
        output_x = Model.extractHiddenState(extractLayer,i,sentences)
#        
        output_x = np.concatenate((output_x[0],output_x[1]),axis=1)
        output.append(output_x)
    
    if len(output) == 2:
        print("combine two model results")
        output = np.concatenate((output[0],output[1]),axis = 1)
        
    elif len(output) == 3:
        print("combine three model results")
        output_temp = np.concatenate(output[0],output[1],axis = 1)
        output = np.concatenate((output[2],output_temp),axis = 1)
    
    else: 
        print("Single model results")
        output = output[0]
    
    return output


In [ ]:
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 215,
                                 'tenacity': 5, 'epoch_size': 4}
# Set up logger
se = senteval.engine.SE(params_senteval, batcher, prepare)
transfer_tasks = ['CR','TREC']
# 'MR', 'CR' ,'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness'  ,'TREC','MRPC'
results = se.eval(transfer_tasks)
print(results)


In [6]:
results

{'CR': {'devacc': 76.55, 'acc': 75.5, 'ndev': 3775, 'ntest': 3775},
 'TREC': {'devacc': 77.2, 'acc': 79.2, 'ndev': 5452, 'ntest': 500}}